In [1]:
import json
import numpy as np
import pandas as pd

In [14]:
def solve():
    global iterations, solutions
    n_lim = 4
    l_lim = 4
    for i, pos in enumerate(posisiton_list):
        if nation_list[i] == -1:
            (nation_set, league_set, club_sets, club_leagues) = get_possible_NLC(i)
            for nat in nation_set:
                for lea in league_set:
                    if lea == 80:
                        continue
                    club_set = club_sets[club_leagues.index(lea)]
                    for clu in club_set:
                        nation_list[i] = nat
                        league_list[i] = lea
                        club_list[i] = clu
                        
                        change_links(i)
                        
                        count_list = [0] * len(posisiton_list)
                        n_count, count_list = nation_count(i, 0, count_list)
                        
                        count_list = [0] * len(posisiton_list)
                        l_count, count_list = league_count(i, 0, count_list)
                        
                        iterations += 1
#                         print(iterations)
#                         print(pos_list)
#                         print(nation_list, "Nations")
#                         print(league_list, "Leagues")
#                         print(club_list, "Clubs")
#                         print(chemistry_listx)

                        if calc_constrain(n_count, l_count, n_lim, l_lim):
                            cont = solve()
                            if cont == 1:
                                return 1
            chemistry_list[i] = 1
            chemistry_listx[i] = 3
            link_matrix[pos["connection_list"], i] = 3
            nation_list[i] = -1
            league_list[i] = -1
            club_list[i] = -1       
            return 0
    solutions += 1
    nations = count_nations()
    leagues = count_leagues()
    clubs = count_clubs()
    print("----", solutions, "----", iterations, "----")
#     print(link_matrix)
#     print(pos_list)
#     print(nation_list, "Nations")
#     print(league_list, "Leagues")
#     print(club_list, "Clubs")
#     print(nations, leagues)
    
    save_list = ([d["formations"][formation_index]['name'], n_lim, l_lim, nations, leagues, clubs, nations+leagues+clubs] + pos_list + 
                 nation_list + league_list + club_list)
    df.loc[len(df), :] = save_list
    
#     a = input("More (y)? ")
#     if a != "y":
#     return 1
#     if solutions == 2:
#         a = input("More (y)? ")
#         if a != "y":
#             return 1
#         return 1
    return 0

In [3]:
def calc_constrain(n_count, l_count, n_lim, l_lim):
    calc_chemistry()
    if sum(chemistry_list) == len(chemistry_list):
        if n_count < n_lim:
            if l_count < l_lim:
#                 print(chemistry_list)
                return True
    return False

In [4]:
def nation_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] == nation_list[idx] and count_list[conn] == 0:
            count, count_list = nation_count(conn, count, count_list)
    return count, count_list

def league_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if league_list[conn] == league_list[idx] and count_list[conn] == 0:
            count, count_list = league_count(conn, count, count_list)
    return count, count_list

def club_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if club_list[conn] == club_list[idx] and league_list[conn] == league_list[idx] and count_list[conn] == 0:
            count, count_list = club_count(conn, count, count_list)
    return count, count_list

In [5]:
def count_nations():
    n_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            n_c += 1
            (c, count_list) = nation_count(i, 0, count_list)
    return n_c

def count_leagues():
    l_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            l_c += 1
            (c, count_list) = league_count(i, 0, count_list)
    return l_c

def count_clubs():
    c_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            c_c += 1
            (c, count_list) = club_count(i, 0, count_list)
    return c_c

In [6]:
def change_links(idx):
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] != -1:
            link = 0
            if nation_list[conn] == nation_list[idx]:
                link += 1
            if league_list[conn] == league_list[idx]:
                link += 1
                if club_list[conn] == club_list[idx]:
                    link += 1
            link_matrix[idx, conn] = link
            link_matrix[conn, idx] = link

def calc_chemistry():
    for i, pos in enumerate(posisiton_list):
        chemistry_listx[i] = sum(link_matrix[i, :]) / len(posisiton_list[i]["connection_list"])
        chemistry_list[i] = min(1, chemistry_listx[i])
        
    
#             chemistry_listx[conn] = sum(link_matrix[conn, :]) / len(posisiton_list[conn]["connection_list"])
#             chemistry_list[conn] = min(1, chemistry_listx[conn])
#     print(link_matrix)
#     print(chemistry_listx)

In [7]:
def get_possible_NLC(idx):
    nation_set = set()
    league_set = set()
    club_leagues = []
    club_sets = []
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] != -1:
            nation_set.add(nation_list[conn])
            league_set.add(league_list[conn])
            if league_list[conn] in club_leagues:
                club_sets[club_leagues.index(league_list[conn])].add(club_list[conn])
            else:
                club_leagues.append(league_list[conn])
                club_sets.append({club_list[conn]})
    
    for i, s in enumerate(club_sets):
        club_sets[i] = s
        club_sets[i].add(max(club_sets[i]) + 1)
        
    if not nation_set:
        nation_set.add(0)
    else:
        nation_set.add(max(nation_set) + 1)

    if not league_set:
        league_set.add(0)
        club_leagues.append(0)
        club_sets.append({0})
    else:
        league_set.add(max(league_set) + 1)
        club_leagues.append(max(league_set))
        club_sets.append({0})
    return (nation_set, league_set, club_sets, club_leagues)

In [17]:
formation_index = 8

f = open("../files/formations.json")
d = json.load(f)
posisiton_list = d["formations"][formation_index]["positions_list"]
name = d["formations"][formation_index]['name']
n_pos = len(posisiton_list)
pos_list = []
for i in posisiton_list:
    if 'side' in i:
        pos_list.append(i['side'] + i['position'])
    else:
        pos_list.append(i['position'])
Pcol = []
Ncol = []
Lcol = []
Ccol = []
for i in range(n_pos):
    Pcol.append('pos'+str(i).zfill(2))
    Ncol.append('N'+str(i).zfill(2))
    Lcol.append('L'+str(i).zfill(2))
    Ccol.append('C'+str(i).zfill(2))
col = ['formation', 'Nlim', 'Llim', 'N', 'L', 'C', 'NLC'] + Pcol + Ncol + Lcol + Ccol
df = pd.DataFrame(columns = col)
print(name)

4-3-3


In [18]:
chemistry_list = [1] * len(posisiton_list)
chemistry_listx = [3] * len(posisiton_list)
nation_list = [-1] * len(posisiton_list)
league_list = [-1] * len(posisiton_list)
club_list = [-1] * len(posisiton_list)
link_matrix = np.zeros((n_pos,n_pos), dtype=int)
for i, pos in enumerate(posisiton_list):
    link_matrix[pos["connection_list"], i] = 3
    

nation_list[2] = 80
league_list[2] = 80
club_list[2] = 0
link_matrix[2,3] = 2
nation_list[3] = 80
league_list[3] = 80
club_list[3] = 1
link_matrix[3,2] = 2
   
iterations = 0
solutions = 0
solve()

# filename = "C:/Users/Truls/OneDrive/Documents/02 - Privat/Hobby/Python/FUT/hybrid_formations/" + name + ".h5"
# df.to_hdf(filename, key = 'df', mode = 'w')
# df

---- 1 ---- 67 ----
---- 2 ---- 68 ----
---- 3 ---- 135 ----
---- 4 ---- 136 ----
---- 5 ---- 215 ----
---- 6 ---- 216 ----
---- 7 ---- 283 ----
---- 8 ---- 284 ----
---- 9 ---- 366 ----
---- 10 ---- 367 ----
---- 11 ---- 388 ----
---- 12 ---- 389 ----
---- 13 ---- 519 ----
---- 14 ---- 520 ----
---- 15 ---- 602 ----
---- 16 ---- 603 ----
---- 17 ---- 670 ----
---- 18 ---- 671 ----
---- 19 ---- 1830 ----
---- 20 ---- 1831 ----
---- 21 ---- 1852 ----
---- 22 ---- 1853 ----
---- 23 ---- 1885 ----
---- 24 ---- 1886 ----
---- 25 ---- 1992 ----
---- 26 ---- 1993 ----
---- 27 ---- 2074 ----
---- 28 ---- 2075 ----
---- 29 ---- 2096 ----
---- 30 ---- 2097 ----
---- 31 ---- 2129 ----
---- 32 ---- 2130 ----


0

In [19]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df

,formation,Nlim,Llim,N,L,C,NLC,pos00,pos01,pos02,pos03,pos04,pos05,pos06,pos07,pos08,pos09,pos10,N00,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,L00,L01,L02,L03,L04,L05,L06,L07,L08,L09,L10,C00,C01,C02,C03,C04,C05,C06,C07,C08,C09,C10
0,4-3-3,4,4,4,4,5,13,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,1,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,0,0,0,0,0
1,4-3-3,4,4,4,4,6,14,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,1,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,0,0,0,0,1
2,4-3-3,4,4,4,4,5,13,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,2,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,0,0,0,0,0
3,4-3-3,4,4,4,4,6,14,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,2,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,0,0,0,0,1
4,4-3-3,4,4,4,4,6,14,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,1,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,1,0,0,0,0
5,4-3-3,4,4,4,4,7,15,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,1,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,1,0,0,0,1
6,4-3-3,4,4,4,4,6,14,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,2,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,1,0,0,0,0
7,4-3-3,4,4,4,4,7,15,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,1,2,2,2,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,1,0,0,0,1
8,4-3-3,4,4,5,4,5,14,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,2,2,2,3,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,0,0,0,0,0
9,4-3-3,4,4,5,4,6,15,GK,RCB,RB,RCM,CM,LCB,LB,LCM,LW,ST,RW,0,0,80,80,0,1,2,2,2,3,80,0,81,80,80,81,0,0,81,82,82,82,0,0,0,1,0,0,0,0,0,0,1


In [28]:
# s = set()
# for index, row in df.iterrows():
l = df[Ncol].values.tolist()
s = set(tuple(i) for i in l)
l1 = list(s)
print(*l1, sep="\n")

(0, 0, 80, 80, 0, 1, 2, 2, 3, 3, 80)
(0, 0, 80, 80, 0, 1, 2, 2, 2, 3, 80)
(0, 0, 80, 80, 0, 1, 2, 3, 3, 3, 80)
(0, 81, 80, 80, 81, 81, 82, 82, 82, 83, 80)
(0, 81, 80, 80, 81, 81, 82, 83, 83, 83, 80)
(0, 81, 80, 80, 81, 81, 82, 82, 83, 83, 80)
(0, 0, 80, 80, 0, 1, 1, 2, 2, 2, 80)
(0, 0, 80, 80, 0, 1, 1, 1, 2, 2, 80)


Add results to pandas<br>
one row for each result<br>
three coloumns for each position (N, L, C) (33 in total)<br>
count unique N, L and C, and add these as coloumns<br>
- how to count clubs?